In [23]:
from query_reader import QueryReader 
import pandas as pd
import psycopg2.extras
import psycopg2
import time
import os

query_dir = os.path.join(os.path.abspath(''), 'queries')
qr = QueryReader(query_dir)
BASE_DATE = "'2021-05-01'"

def create_connection(): 
    return psycopg2.connect(
        dbname='main',
        host='main-us-e2.cmbsiiqeauby.us-east-2.rds.amazonaws.com',
        port=5432,
        user='doadmin',
        password='i39kew8n7jcat7l9'
    )

def run_query(connection, query, cursor=None):
    if cursor:
        cursor = connection.cursor(cursor_factory = psycopg2.extras.RealDictCursor)
    else:
        cursor = connection.cursor()
    try:
        start_time = time.time()
        cursor.execute(query)
        results = cursor.fetchall()
        end_time = time.time()

        if len(results[0]) == 1:
            results = [x[0] for x in results]
        else:
            results = [x for x in results]
        print(f'Time elapsed: {end_time - start_time}')
        print(f'Length of Results: {len(results)}')
        print(f'Sample Result: {results[0]}')
        return results
    except:
        print('rolling back')
        connection.rollback()
    finally:
        cursor.close()

def big_fetch(connection, query, cursor=None):
    if cursor:
        cursor = connection.cursor('big_fetch_cursos', cursor_factory = cursor)
    else:
        cursor = connection.cursor('big_fetch_crusor')
    results = []
    start_time = time.time()
    cursor.execute(query)
    while True:
        rows = cursor.fetchmany(5000)
        if not rows:
            break
        results.extend(rows)

    end_time = time.time()
    print(f'Time elapsed: {end_time - start_time}')
    print(f'Length of Results: {len(results)}')
    print(f'Sample Result: {results[0]}')
    cursor.close()
    return results


def read_into_df(connection, query) -> pd.DataFrame:
    start_time = time.time()
    df = pd.read_sql(query, connection)
    end_time = time.time()
    print(f'Time elapsed: {end_time - start_time}')
    print(f'DF Shape: {df.shape}')
    return df



conn = create_connection()

In [6]:
raw_map_ids_query = qr.read_query('map_id.sql')
map_ids_query = raw_map_ids_query.format(date=BASE_DATE) 
map_ids = run_query(conn, map_ids_query)
str_map_ids = ','.join([str(x) for x in map_ids])

Time elapsed: 0.8576109409332275
Length of Results: 11897
Sample Result: 2


In [7]:
raw_ck2 = qr.read_query('csgo_kills.sql')
ck2 = raw_ck2.format(date=BASE_DATE)
results = run_query(conn, ck2)

Time elapsed: 19.919265031814575
Length of Results: 53828
Sample Result: (0, 76561197960710573, 16, Decimal('16.0000'), Decimal('1.0000'), Decimal('1.0000'), Decimal('9.0000'), Decimal('9.0000'), 6, Decimal('6.0000'), 0, Decimal('0.0000'), 0, Decimal('0.0000'), 0, Decimal('0.0000'), 0, Decimal('0.0000'), 1, Decimal('1.0000'), 9, Decimal('9.0000'))


In [21]:

raw_econ = qr.read_query('econ_rebuild.sql')
econ = raw_econ.format(date=BASE_DATE)
results = big_fetch(conn, econ)

Time elapsed: 162.8395390510559
Length of Results: 2800056
Sample Result: RealDictRow([('match_id', 0), ('map_id', 2), ('round_number', 1), ('team_name', 'FURIA'), ('steam_id', 76561197965021087), ('won', 0), ('is_tside', 1), ('round_end_reason', 'BombDefused'), ('rifle_kills', 0), ('sniper_kills', 0), ('grenade_kills', 0), ('lmg_kills', 0), ('shotgun_kills', 0), ('smg_kills', 0), ('pistol_kills', 0), ('melee_kills', 0), ('awp_kills', 0), ('p90_kills', 0), ('cz75_kills', 0), ('bomb_plants', 0), ('bomb_defused', 0), ('bomb_exploded', 0)])


In [24]:
raw_econ = qr.read_query('econ_rebuild.sql')
econ = raw_econ.format(date=BASE_DATE)
results = run_query(conn, econ, cursor=psycopg2.extras.RealDictCursor)

Time elapsed: 143.6016550064087
Length of Results: 2800056
Sample Result: RealDictRow([('match_id', 0), ('map_id', 2), ('round_number', 1), ('team_name', 'FURIA'), ('steam_id', 76561197965021087), ('won', 0), ('is_tside', 1), ('round_end_reason', 'BombDefused'), ('rifle_kills', 0), ('sniper_kills', 0), ('grenade_kills', 0), ('lmg_kills', 0), ('shotgun_kills', 0), ('smg_kills', 0), ('pistol_kills', 0), ('melee_kills', 0), ('awp_kills', 0), ('p90_kills', 0), ('cz75_kills', 0), ('bomb_plants', 0), ('bomb_defused', 0), ('bomb_exploded', 0)])


In [25]:
raw_econ = qr.read_query('econ_rebuild.sql')
econ = raw_econ.format(date=BASE_DATE)
results = run_query(conn, econ)

Time elapsed: 101.27957320213318
Length of Results: 2800056
Sample Result: (0, 2, 1, 'FURIA', 76561197965021087, 0, 1, 'BombDefused', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)


In [15]:
from econ_rebuilder import run_econ_rebuild

raw_econ = qr.read_query('econ_rebuild.sql')
econ = raw_econ.format(date=BASE_DATE)
indf = read_into_df(conn, econ)



/Users/matias/.local/share/virtualenvs/ModelNotebooks-OWhv5NqQ/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Time elapsed: 94.80610990524292
DF Shape: (2800056, 22)


In [16]:
df.to_csv('data/base_econ_df.csv', index=False)

In [17]:

econ_df = run_econ_rebuild(df)
econ_df.head()

Finished streaks, time taken: 34.43226218223572
Finished groupping, time taken: 77.0827112197876
Finished round end bonus, time taken: 183.50504326820374
Finished aplpying weapon bonus, time taken: 5.074497699737549
moneydf: (110480, 4)
match_money: (53828, 4)
Finished aplpying weapon bonus, time taken: 3.6867661476135254


,match_id,steam_id,total_money_per_map,total_money
0,0,76561197960710573,68900.0,68900
1,0,76561197965021087,61900.0,61900
2,0,76561197996370184,58600.0,58600
3,0,76561198013243326,70500.0,70500
4,0,76561198016432560,70200.0,70200


In [11]:
econ_df.to_csv('output/econ_rebuild.csv')